# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [3]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [4]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-30'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2520,2022-11-25,"$16,602.27","$16,603.32","$16,388.40","$16,521.84","$16,521.84",18678255976
2521,2022-11-26,"$16,521.58","$16,666.86","$16,416.23","$16,464.28","$16,464.28",18000008764
2522,2022-11-27,"$16,463.88","$16,594.41","$16,437.03","$16,444.63","$16,444.63",20443898509
2523,2022-11-28,"$16,440.22","$16,482.93","$16,054.53","$16,217.32","$16,217.32",27743025156
2524,2022-11-29,"$16,217.64","$16,522.26","$16,139.40","$16,444.98","$16,444.98",23581685468


In [7]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [8]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)
df

,ds,y
0,2016-01-01,$430.72
1,2016-01-02,$434.62
2,2016-01-03,$433.58
3,2016-01-04,$430.06
4,2016-01-05,$433.07
...,...,...
2520,2022-11-25,"$16,602.27"
2521,2022-11-26,"$16,521.58"
2522,2022-11-27,"$16,463.88"
2523,2022-11-28,"$16,440.22"


In [12]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      2525 non-null   datetime64[ns]
 1   y       2525 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 39.6 KB


In [15]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Previsión Bitcoin"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [16]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')


In [17]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [20]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.head()

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [22]:
# Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [23]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2016-01-01,$199.94,"$-2,832.19","$3,336.04",$199.94,$199.94,$0.14,$0.14,$0.14,$-0.00,$-0.00,$-0.00,$0.14,$0.14,$0.14,$0.00,$0.00,$0.00,$227.65
1,2016-01-02,$201.76,"$-3,097.32","$2,942.14",$201.76,$201.76,$0.14,$0.14,$0.14,$-0.00,$-0.00,$-0.00,$0.14,$0.14,$0.14,$0.00,$0.00,$0.00,$230.61
2,2016-01-03,$203.59,"$-2,662.37","$3,500.50",$203.59,$203.59,$0.15,$0.15,$0.15,$-0.00,$-0.00,$-0.00,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,$233.35
3,2016-01-04,$205.41,"$-3,047.17","$3,428.90",$205.41,$205.41,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,$235.82
4,2016-01-05,$207.24,"$-2,724.18","$3,393.68",$207.24,$207.24,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,$237.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2023-11-25,"$-13,319.85","$-43,874.96","$12,181.87","$-35,623.47","$9,912.37",$0.22,$0.22,$0.22,$-0.00,$-0.00,$-0.00,$0.22,$0.22,$0.22,$0.00,$0.00,$0.00,"$-16,276.56"
2886,2023-11-26,"$-13,400.73","$-44,375.14","$11,146.62","$-35,754.82","$9,900.97",$0.22,$0.22,$0.22,$-0.00,$-0.00,$-0.00,$0.22,$0.22,$0.22,$0.00,$0.00,$0.00,"$-16,297.50"
2887,2023-11-27,"$-13,481.60","$-43,835.46","$11,874.36","$-35,959.60","$9,889.57",$0.21,$0.21,$0.21,$0.00,$0.00,$0.00,$0.21,$0.21,$0.21,$0.00,$0.00,$0.00,"$-16,307.74"
2888,2023-11-28,"$-13,562.48","$-44,703.49","$12,250.48","$-36,177.35","$9,877.66",$0.20,$0.20,$0.20,$0.00,$0.00,$0.00,$0.20,$0.20,$0.20,$0.00,$0.00,$0.00,"$-16,303.39"


In [24]:
forecast =forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast.tail()

,ds,yhat,yhat_lower,yhat_upper
2885,2023-11-25,"$-16,276.56","$-43,874.96","$12,181.87"
2886,2023-11-26,"$-16,297.50","$-44,375.14","$11,146.62"
2887,2023-11-27,"$-16,307.74","$-43,835.46","$11,874.36"
2888,2023-11-28,"$-16,303.39","$-44,703.49","$12,250.48"
2889,2023-11-29,"$-16,266.49","$-43,975.04","$10,937.67"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [25]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)